In [14]:
#impport modules
import requests
import re
from bs4 import BeautifulSoup
import os
import pandas as pd
import time
from urllib.request import quote



#get the html text from the webpage
def getHTMLText(url):
    try:
        r=requests.get(url,timeout=30)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        html=r.text
        print('html is ready')
        return html
    except:
        print('get html text failed')

#get the code of the city 
def citycoder(city):
    url='https://js.51jobcdn.com/in/js/2016/layer/area_array_c.js'
    r= requests.get(url,timeout=30)
    fl=r.text
    geocode = re.findall('"([0-9]+)":"{}"'.format(city),fl)[0]
    return geocode

#soup the html file
def soupmaker(html):
    soup = BeautifulSoup(html,'html.parser')
    return soup 
    print('soup is ready')

#extract data we want from the soup
def listfiller(tags): #get the information we need
    jobtitle=list()
    joburl=list()
    company=list()
    location=list()
    salary=list()
    for tag in tags:
        try:
            if 't1' in tag.get('class',None):
                jobtitle.append(tag.find('a').string.strip())
                joburl.append(tag.find('a').get('href',None))
            if 't2' in tag.get('class',None):
                company.append(tag.string)
            if 't3' in tag.get('class',None):
                location.append(tag.string)
            if 't4' in tag.get('class',None):
                salary.append(tag.string)
            else:
                continue
        except:
            continue
    company=company[1:]
    location=location[1:]
    salary=salary[1:]
    print('list has been filled')
    return jobtitle,joburl,company,location,salary




jobtitle=[]
joburl=[]
company=[]
location=[]
salary=[]

kw = quote(input('职位名称').strip())
dump=int(int(input('爬取多少页').strip())+1)
geocode=citycoder(str(input('城市').strip()))
for page in range(1,dump):
    url='https://search.51job.com/list/{},000000,0000,00,9,99,{},2,{}.html'.format(geocode,kw,page)
    print(url)
    html = getHTMLText(url)
    soup = soupmaker(html)
    
    tags=soup(class_=['t1','t2','t3','t4'])
    L=listfiller(tags)

    jobtitle += L[0]
    joburl+=L[1]
    company+=L[2]
    location+=L[3]
    salary+=L[4]

    time.sleep(2) # Slow things down so as to not hammer Wikipedia's servers




raw = {
    'jobtitle' : jobtitle,
    'company': company,
    'location': location,
    'salary': salary,
    'link': joburl
}

df = pd.DataFrame(raw)
print('dataframe has been created')

df.head(10)


https://search.51job.com/list/020000,000000,0000,00,9,99,%E6%95%B0%E6%8D%AE%E8%BF%90%E8%90%A5,2,1.html
html is ready
list has been filled
https://search.51job.com/list/020000,000000,0000,00,9,99,%E6%95%B0%E6%8D%AE%E8%BF%90%E8%90%A5,2,2.html
html is ready
list has been filled
https://search.51job.com/list/020000,000000,0000,00,9,99,%E6%95%B0%E6%8D%AE%E8%BF%90%E8%90%A5,2,3.html
html is ready
list has been filled
https://search.51job.com/list/020000,000000,0000,00,9,99,%E6%95%B0%E6%8D%AE%E8%BF%90%E8%90%A5,2,4.html
html is ready
list has been filled
https://search.51job.com/list/020000,000000,0000,00,9,99,%E6%95%B0%E6%8D%AE%E8%BF%90%E8%90%A5,2,5.html
html is ready
list has been filled
dataframe has been created


,jobtitle,company,location,salary,link
0,数据运营主管（财务方向）,上海贝锐信息科技股份有限公司,上海-杨浦区,1.2-1.5万/月,https://jobs.51job.com/shanghai-ypq/120712933....
1,数据运营专员,上海走起网络科技有限公司,上海-长宁区,0.7-1万/月,https://jobs.51job.com/shanghai-cnq/119956871....
2,数据运营主管,上海贝锐信息科技股份有限公司,上海,1-1.5万/月,https://jobs.51job.com/shanghai/120860369.html...
3,数据运营专员,全联保险经纪有限公司,上海-杨浦区,6-8千/月,https://jobs.51job.com/shanghai-ypq/116582576....
4,数据运营专员,上海量讯通信科技有限公司,上海-徐汇区,4.5-6千/月,https://jobs.51job.com/shanghai-xhq/120733534....
5,运营管理（数据运营方向）,上海德佑物业顾问有限公司,上海-静安区,6-8千/月,https://jobs.51job.com/shanghai-jaq/117363581....
6,数据运营和分析工程师-上海-00286,上海领健信息技术有限公司,上海,1.5-2.5万/月,https://jobs.51job.com/shanghai/121086752.html...
7,电子商务数据运营(000932)（上海）,长虹・美菱中国区,上海,4.5-6千/月,https://jobs.51job.com/shanghai/116343708.html...
8,数据运营（抖音方向）,上海弘亚文化传媒有限公司,上海-宝山区,6-8千/月,https://jobs.51job.com/shanghai-bsq/118288474....
9,数据运营4500（酒店）,上海佩琪信息技术有限公司,上海,4-6千/月,https://jobs.51job.com/shanghai/119800308.html...
